In [1]:
import tensorflow as tf

2022-07-18 18:44:10.218893: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-18 18:44:10.218981: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
print(tf.__version__)

2.7.0


In [3]:
import requests
import pandas as pd
import numpy as np

In [4]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/TreinoRedes/obtemListaParaPrevisao"

In [5]:
urlExemplos = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploTreinoAcaoEntradas/obtemProximoDia"

In [6]:
r = requests.get(url)
j = r.json()
dfTreino = pd.DataFrame.from_dict(j)

In [7]:
dfTreino.head()

,id,grupoAcaoId,periodoTreinoRedeId,tipoExemploTreinoId,regraProjecaoId,redeNeuralId,ativoPrevisao,tipoCompraVenda
0,2,12,1,1,30,2,1,V


In [8]:
global dfDados
r = requests.get(urlExemplos)
j = r.json()
dfDados = pd.DataFrame.from_dict(j)

In [9]:
dfDados.head()

,campoX,ticker,diaNumPrevisao
0,"0.989719, 0.991090, 0.988348, 0.997944, 1.009...",ABEV3,20220718
1,"2.215328, 2.248175, 2.237226, 2.237226, 2.229...",AERI3,20220718
2,"1.656285, 1.674638, 1.679644, 1.674638, 1.669...",ALPA4,20220718
3,"1.305296, 1.319626, 1.315888, 1.323364, 1.315...",ALSO3,20220718
4,"1.516588, 1.540284, 1.568720, 1.559242, 1.554...",AMAR3,20220718


In [10]:
global dfEntradas

In [11]:
def incluiEntrada(treino):
    urlEntrada = "http://vps-40d69db1.vps.ovh.ca:23004/api/CotacaoIntradayAcaoResultados/listaPrecoEntradaPrevisaoProximoB3?idRegra=" + str(treino['regraProjecaoId']) + "&idGrupo=" + str(treino['grupoAcaoId'])
    print(urlEntrada)
    r = requests.get(urlEntrada)
    j = r.json()
    dfEntradas = pd.DataFrame.from_dict(j)
    print(dfEntradas.head())
    return dfEntradas
  

In [12]:
def trataRede(treino):
    print('ID:' , treino['id'])
    nome_arquivo = 'pesos/treino_rede' + str(treino['id']) + '.h5'
    print(nome_arquivo)
    modelo = tf.keras.models.load_model(nome_arquivo)
    modelo.summary()
    result = modelo.predict(X)
    dfDados['resultado'] = result
    dfDados['treinoRedeId'] = str(treino['id'])
    dfDados['tipoCompraVenda'] = treino['tipoCompraVenda']
    dfEntradas = incluiEntrada(treino)
    dfPrevisao = pd.merge(dfDados,dfEntradas, how='left', left_on='ticker' ,  right_on='ticker')
    print('ticker previsão:' , len(dfPrevisao))
    json = dfPrevisao.to_dict(orient='list')
    urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/PrevisaoRedes/recebePrevisaoTreino'
    print(json)
    response = requests.post(urlPrevisao, data=json)
    print(response)

In [13]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [14]:
dfTratadoX = dfDados.apply(trataLinhaX, axis='columns')
X = np.stack(dfTratadoX.values)
X.shape

(87, 1800)

In [15]:
dfDados.drop('campoX', inplace=True, axis=1)

In [16]:
dfTreino.apply(trataRede, axis='columns')

ID: 2
pesos/treino_rede2.h5


2022-07-18 18:46:12.677980: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-18 18:46:12.689548: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vps34755): /proc/driver/nvidia/version does not exist
2022-07-18 18:46:12.816611: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-18 18:46:17.319531: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12960000 exceeds 10% of free system memory.
2022-07-18 18:46:18.378622: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 12960000 exceeds 10% of free system memory.
2022-07-18 18:46:

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1800)              3241800   
                                                                 
 dense_1 (Dense)             (None, 1400)              2521400   
                                                                 
 dense_2 (Dense)             (None, 1000)              1401000   
                                                                 
 dense_3 (Dense)             (None, 600)               600600    
                                                                 
 dense_4 (Dense)             (None, 200)               120200    
                                                                 
 dense_5 (Dense)             (None, 1)                 201       
                                                                 
Total params: 7,885,201
Trainable params: 7,885,201
Non-

0    None
dtype: object